# Vector Databases

## Why Vector Databases?

Vector data are high-dimensional and traditional dbs are not built to efficiently store and retrieve vectors. Because of this: Vector DBs are designed to store and retrieve vector data - (duh). 

## Linear Algebra 101

### Vectors

<img src="https://www.illumination.com/wp-content/uploads/2019/11/DM1_Vector.png" width="250"/>

Vector: **Direction + Magnitude**

* collection of numbers

* can represent different things (**embedding**)
    - language
    - images
    - audio
* High School Cliques Analogy
* <img src="https://miro.medium.com/v2/resize:fit:720/format:webp/1*dyH20eCqb6qTL-gt4nCVzQ.png" width="700"/>


* Applications
    - text generation
    - recommendation systems
    - search engines

### **Embeddings == Vectors**
(but Vector doesn't necessarily mean embedding)

### VectorDB
* used to store/query these embeddings
* arrays of numbers clustered
    - relational db: rows/columns
    - document db: documents/collections


# Simple VectorDB implementation in LangChain

First, we install the necessary packages.

`langchain` is a framework for using anything related utilizing Large Language Models (LLMs).

`sentence-transformers` is required to utilize HuggingFace's Embeddings.

`faiss-cpu`: FAISS is a vector DB that will be used in this tutorial.

`pypdf`: required package for the 'PDFLoader' we will use - used to read text from PDFs.



In [ ]:
%%capture

!pip install langchain
!pip install sentence-transformers
!pip install torch
!pip install faiss-cpu
!pip install pypdf

!pip install scikit-learn

!pip install spacy
!python -m spacy download en_core_web_lg

If using Google Colab, you need to download the sample files shown in this tutorial:

In [ ]:
!wget -O files.zip https://github.com/KoltonHauck/BMI6016_VectorDB/raw/main/files.zip

!unzip files.zip -d .

Now we can import everything we will use.

`PyPDFDirectoryLoader` is a 'document loader', which means it processes a folder with .pdfs and extracts the text from them. All of the different loader formats langchain implementations are here: [LangChain Loaders](https://python.langchain.com/docs/integrations/document_loaders)

`RecursiveCharacterTextSplitter` is a 'text splitter': it takes in 'document loader' text documents and splits the documents in manageable chunks. Chunking is important for several reasons:
1. size limitations of embedding models
2. search precision -> when entire docs encoded as single vectors: specificity of embeddings may decrease
3. memory efficiency -> processing chunks is computationally cheaper than processing whole documents
4. parallel processing -> can process chunks in parallel

LangChain text splitters found here: [Text Splitters](https://python.langchain.com/docs/modules/data_connection/document_transformers/)

`HuggingFaceEmbeddings`: used to generate the embeddings for the text chunks. (natural language -> vector representation) (The default model selected is [`all-mpnet-base-v2`](https://huggingface.co/sentence-transformers/all-mpnet-base-v2)) This is just the example used in this example. There are many ways to generate embeddings (just a few):
* one hot encoding
* word2vec
* GloVe
* BERT (transformer)

`FAISS`: in-memory vector DB used in this tutorial.

In [ ]:
import langchain
from langchain.document_loaders import PyPDFDirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS

In [ ]:
# load pdfs using PyPDFDirectoryLoader

loader = PyPDFDirectoryLoader("files/pdfs/")
docs = loader.load()
len(docs)

In [ ]:
docs[0]

In [ ]:
len(docs[0].page_content)

In [ ]:
# split text into chunks
# chunk overlap: some text is shared between adjacent chunks
# important for context preservation, continuity in search results, reducing boundary effects

text_splitter = RecursiveCharacterTextSplitter(chunk_size=512, chunk_overlap=64)
texts = text_splitter.split_documents(docs)
len(texts)

In [ ]:
texts[0]

In [ ]:
len(texts[0].page_content)

In [ ]:
# peek at first 'text document'
print(texts[0].page_content)

In [ ]:
# init embeddings model
# text -> vector

import torch

# Determine if a GPU is available and choose the appropriate device
device = "cuda" if torch.cuda.is_available() else "cpu"

embeddings = HuggingFaceEmbeddings(
    model_name="thenlper/gte-large",
    model_kwargs={"device": device},
    encode_kwargs={"normalize_embeddings": True},
)

In [ ]:
# generate embeddings

query_result = embeddings.embed_query(texts[0].page_content)

# the length of texts[0].page_content --> 268
# embeddings length --> 1024
print(len(query_result))

In [ ]:
# it is now just a list / array of numbers

query_result[:20]

Each of the 'texts' is now a point in high-dimensional space (1024D space). Similar texts will be closer together in this high-dimensional space.

We can now create a Vector Database from these texts using FAISS.

In [ ]:
# may take several minutes if on CPU
# if on cpu, suggest reducing 'texts' being passed in: eg texts[:100]
# once created, this is living 'in memory', but can be saved to hard drive if desired

vector_db = FAISS.from_documents(texts, embeddings)

With the VectorDB created, we can now do some pretty cool things with it.

## Basic Similarity Search

With the `.similarity_search()` method, we can extract documents (`texts`) from the vector DB that are similar to the query. The query gets embedded, and similar vectors to the query vector are retrieved. Here we are using the `.similarity_search_with_score()` method which is essentially the same, but also provides the `similarity score` between the query and retrieved text. The lower the number, the more similar!

The `k` parameter is the number of `texts` to retrieve from the vector DB

In [ ]:
sim_search = vector_db.similarity_search_with_score("What are some frameworks to assess data quality?", k=4)

sim_search

In [ ]:
for i, result in enumerate(sim_search):
  print(f"---- Result #{i} | {result[0].metadata['source']} | page {result[0].metadata['page']} | score: {result[1]} ----")
  print(result[0].page_content, "\n")

## Max Marginal Relevance (MMR) Search

MMR is a search algorithm that attempts to address the limitations of basic similarity search:
* redundancy (very similar documents)
* coverage (when searching for 'apple': fruit or computer? MMR might return documents relevant to both whereas basic might just return one)
* narrow coverage of topic (MMR helps to provide comprehensive view of topic)

MMR works by:
* calculating relevance scores between query and each document (similar to basic search)
* iteratively selecting documents based on similarity to the query AND dissimilarity to already selected documents (can tune with parameter `lambda_mult`)

Implemented with `max_marginal_relevance_search` method.

In [ ]:
# lambda_mult = 1 (basically basic search) -> takes into no consideration of dissimilarity of already retrieved texts

mmr_result_1 = vector_db.max_marginal_relevance_search("What are some frameworks to assess data quality?", k=4, lambda_mult=1)

for i, result in enumerate(mmr_result_1):
  print(f"---- Result #{i} | {result.metadata['source']} | page {result.metadata['page']} ----")
  print(result.page_content, "\n")

In [ ]:
# lambda_mult = 0 -> wildly takes into consideration of dissimilarity of already retrieved texts

mmr_result_0 = vector_db.max_marginal_relevance_search("What are some frameworks to assess data quality?", k=4, lambda_mult=0)

for i, result in enumerate(mmr_result_0):
  print(f"---- Result #{i} | {result.metadata['source']} | page {result.metadata['page']} ----")
  print(result.page_content, "\n")

# Retrieval Augmented Generation (RAG)

RAG is an approach to augment Large Language Models responses by suppling context with the prompt. This helps deal with several issues commonly seen with LLMs:
* hallucinations (by supplying context relevant to the query, the model doesn't need to hallucinate - at least not as much - becuase it has the relevant information needed to ***********)

## Other Embedding Methods

### spaCy

[spaCy](https://spacy.io/) is a great Python NLP package. You can also retrieve embeddings from it!

When you initially install spaCy, it comes pre-loaded with a model packed with a bunch of stuff, however, it does not come pre-loaded with the word vectors. So, we downloaded that right after we 'pip installed' spacy: `!python -m spacy download en_core_web_lg`. We load it initially to retrieve the word vectors.

In [ ]:
# here we are loading the 

import spacy
nlp = spacy.load('en_core_web_lg')

In [ ]:
cheese_emb = nlp.vocab['cheese'].vector # replace cheese

print(len(cheese_emb))

You can't really have a VectorDB with embeddings from two different models / methods. It's like having a dictionary with english and spanish words (but with no translation between them).

### TF-IDF

TF-IDF (Term Frequency-Inverse Document Frequency) is a statistical measure used to evaluate the importance of a word in a document relative to a collection of documents, known as a corpus. It increases proportionally to the number of times a word appears in the document but is offset by the frequency of the word in the corpus. This helps to adjust for the fact that some words appear more frequently in general. TF-IDF is often used in text mining and information retrieval to weigh and rank words' relevance in documents. You can also use TF-IDF embeddings just like other Embeddings shown here.

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Create an instance of TfidfVectorizer
vectorizer = TfidfVectorizer()

# Fit and transform the documents
tfidf_matrix = vectorizer.fit_transform(texts)

# Using Huggingface Models

https://github.com/MuhammadMoinFaisal/LargeLanguageModelsProjects/blob/main/Run%20Llama2%20Google%20Colab/Llama_2_updated.ipynb

https://python.langchain.com/docs/integrations/llms/llamacpp

In [ ]:
%%capture
!CMAKE_ARGS="-DLLAMA_CUBLAS=on" FORCE_CMAKE=1 pip install llama-cpp-python numpy --force-reinstall --upgrade --no-cache-dir --verbose
!pip install llama-cpp-python
!pip install huggingface-hub langchain langchain-community

In [ ]:
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain_community.llms import LlamaCpp

from huggingface_hub import hf_hub_download

In [ ]:
downloaded_model_path = hf_hub_download(repo_id="TheBloke/Llama-2-7b-Chat-GGUF", filename="llama-2-7b-chat.Q5_K_M.gguf")

In [ ]:
# Callbacks support token-wise streaming
callback_manager = CallbackManager([StreamingStdOutCallbackHandler()])

n_gpu_layers = -1  # The number of layers to put on the GPU. The rest will be on the CPU. If you don't know how many layers there are, you can use -1 to move all to GPU.
n_batch = 1024  # Should be between 1 and n_ctx, consider the amount of VRAM in your GPU.

llm = LlamaCpp(
    model_path=downloaded_model_path,
    n_gpu_layers=n_gpu_layers,
    n_batch=n_batch,
    callback_manager=callback_manager,
    verbose=True,  # Verbose is required to pass to the callback manager
)

In [ ]:
prompt = """
Question: A rap battle between Stephen Colbert and John Oliver
"""
llm.invoke(prompt)

In [ ]:
# for just using the llama-cpp package

from llama_cpp import Llama
lcpp_llm = None
lcpp_llm = Llama(
    model_path=downloaded_model_path,
    n_threads=2, # CPU cores
    n_batch=512, # Should be between 1 and n_ctx, consider the amount of VRAM in your GPU.
    n_gpu_layers=32 # Change this value based on your model and your GPU VRAM pool.
    )